# Semianr 11 - Applied Quantitative Logistics

### Harmony Search (HS)

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Sphere Problem

$$
\min{z} = f_{sph}(x) = \begin{equation*}
 \sum_{i=1}^n {x_i}^2 \end{equation*}
$$

$$
x_{min} \le x_i \le x_{max}
$$

Optimal Solutions:

$$
\forall i \;
\left\{
    \begin{array}\\
        x_i^* = 0 \\
        z^* = 0 \\
    \end{array}
\right.
$$

In [ ]:
def sphere(x):
    
    global NFE
    
    if pd.isna(NFE):
        NFE = 0
        
    NFE += 1
    
    z = [item**2 for item in x]
    
    return sum(z)

In [ ]:
# Sort the population and cost (based on the cost)
def pop_sort(p, c):
    li = []
    for i in range(len(c)):
        li.append([c[i],i])
        
    li.sort()
    sort_index = []
    
    for x in li:
        sort_index.append(x[1])
    
    positions, cost = [], []
    for i in sort_index:
        positions.append(p[i])
        cost.append(c[i])
        
    return positions, cost

### HS Algorithm

In [ ]:
## Problem Parameters
nVar = 5      # Number of Decision Variables

varMin = -10  # Decision Variables Lower Bound
varMax = 10   # Decision Variables Upper Bound

global NFE
NFE = 0

nfe = []


## HS Parameters
maxIteration = 1000         # Maximum number of iteration

HMS = 20        # Harmony Memory Size

nNew = 20       # number of New Harmonies

HMCR = 0.5      # Harmony Memory Consideration Rate

PAR = 0.1       # Pitch Adjustment Rate

FW = 0.02*(varMax-varMin)     # Fret Width (bandwidth)

FW_damp = 0.995 # Fret Width Damp Ratio

## Initialization
pop, costs = [], []

# List to store all best costs
BestCosts_list = []

for i in range(HMS):
    pop.append(list(np.random.uniform(varMin, varMax, size=nVar)))
    costs.append(sphere(pop[i]))

# Sort the population and costs
pop, costs = pop_sort(pop, costs)

# Store Best Sol and Cost Ever Found
BestSol = pop[0]
BestSol_Cost = costs[0]


## HS Main Loop
for it in range(maxIteration):
    
    # Initialize List of New Harmonies 
    pop_new, costs_new = [], []
    for k in range(nNew):
        # Use random Value
        pop_new.append(list(np.random.uniform(varMin, varMax, size=nVar)))
    
    for k in range(nNew):
        for j in range(nVar):
            if np.random.random() <= HMCR:
                # Use Harmony Memory
                i = np.random.randint(0, HMS)
                pop_new[k][j] = pop[i][j]
                
            
            # Pitch Adjustment
            if np.random.random() <= PAR:
                DELTA = np.random.uniform(-FW, +FW)
                pop_new[k][j] = pop_new[k][j]+DELTA
                
            
            # Apply Variable Limits
            pop_new[k][j] = max(pop_new[k][j], varMin)
            pop_new[k][j] = min(pop_new[k][j], varMax)
            
        
        # Evaluation
        costs_new.append(sphere(pop_new[k]))
                
    # Merge harmony Memory and New Harmonies
    pop = pop + pop_new
    costs = costs + costs_new
    
    # Sort the population and costs
    pop, costs = pop_sort(pop, costs)
    
    # Truncate Extra Harmonies
    pop = pop[:HMS]
    costs = costs[:HMS]
    
    # Store Best Sol and Cost Ever Found
    BestSol = pop[0]
    BestSol_Cost = costs[0]
    
    # Update the Best Costs for Each Iteration
    BestCosts_list.append(BestSol_Cost)
    
    # store nfe
    nfe.append(NFE)
    
    print(f'Iteration {it} : NFE = {nfe[it]},  Best Cost = {BestCosts_list[it]}')
    
    # Damp Fret Width
    FW = FW*FW_damp

In [ ]:
plt.plot(BestCosts_list, linewidth = 3)
plt.xlabel('NFE')
plt.ylabel('Best Cost')